In [ ]:
import pandas as pd
import os

In [ ]:
train_data = pd.read_csv('ner/twitter17_train.csv')
test_data = pd.read_csv('ner/twitter17_test.csv')
label_col = 'entity_annotations'

In [ ]:
train_data.iloc[0]

In [ ]:
image_col = 'image'
train_data[image_col] = train_data[image_col].apply(lambda ele: ele.split(';')[0]) # Use the first image for a quick tutorial
test_data[image_col] = test_data[image_col].apply(lambda ele: ele.split(';')[0])

def path_expander(path, base_folder):
	path_l = path.split(';')
	p = ';'.join([os.path.abspath(base_folder+path) for path in path_l])
	return p

train_data[image_col] = train_data[image_col].apply(lambda ele: path_expander(ele, base_folder='ner'))
test_data[image_col] = test_data[image_col].apply(lambda ele: path_expander(ele, base_folder='ner'))

train_data[image_col].iloc[0]

In [ ]:
example_row = train_data[label_col].iloc[0]

example_row

In [ ]:
from autogluon.multimodal import MultiModalPredictor

label_col = "entity_annotations"
predictor = MultiModalPredictor(problem_type="ner", label=label_col)
predictor.fit(
	train_data=train_data,
	column_types={"text_snippet":"text_ner"},
)

In [ ]:
prediction_input = test_data.drop(columns=label_col).head(1)
predictions = predictor.predict(prediction_input)
print('Tweet:', prediction_input.text_snippet[0])
print('Image path:', prediction_input.image[0])
print('Predicted entities:', predictions[0])

for entity in predictions[0]:
	print(f"Word '{prediction_input.text_snippet[0][entity['start']:entity['end']]}' belongs to group: {entity['entity_group']}")